In [1]:
import tkinter as tk
from tkinter import filedialog
import tkinter.ttk as ttk
from PIL import ImageTk,Image
import pygame
import pydub
import os
from os import path
from pydub import AudioSegment 
from pydub.playback import play 
import sounddevice as sd
import numpy as np
import os

root=tk.Tk()
root.title("Audio Compresseur")

def browse_button():
    # Allow user to select a file and store it in global var
    # called folder_path
    global file_path
    filename = filedialog.askopenfilename()
    file_path.set(filename)
    
    label1.config(text="Taille D'origine : "+str(os.path.getsize(file_path.get()))+"ko")

def lire_audio(audio):
    nom,extension=os.path.splitext(audio)
    f=AudioSegment.from_file(file=audio,format=extension[1:])
    #ecouter :
    v_data=np.frombuffer(f.raw_data,np.int16)
#     sd.play(v_data,f.frame_rate) 
    
    return(f.channels,f.sample_width,f.frame_rate,f.frame_width,f.duration_seconds,f.raw_data,len(f.raw_data))

def preparer(data1):
    chaine=np.frombuffer(data1,np.int16)
    global n
    for i in range(20):
        if max(chaine)*10**(-i)<=255:
            n=-i
            break
    chaine=[int(i*10**(n)) for i in chaine ]

    chaine=np.around(chaine)
    global indice
    indice=[]
    for i in range(len(chaine)):
        if chaine[i]<=0:
            chaine[i]=int(chaine[i])*(-1)
            indice.append(i)

    string_data=[]
    for i in range(len(chaine)):
        string_data.append(int(chaine[i]).to_bytes(2,'big'))
    return string_data,chaine


def preparer_inv(data2):
    l=np.arange(len(data2))
    for i in range(len(data2)):
        l[i]=data2[i]*(10**(-n))
        if i in indice:
            l[i]=l[i]*(-1)
    return l

def LZWencoding(chaine):
    d={chr(i):i for i in range(0,255)}
    encodeur=[]
    i=len(d)+1
    j=0
    c=str(chaine[j])
    while j!=len(chaine)-1:
        if c not in d.keys():
            d[c]=i
            i+=1
            c=str(chaine[j])
        else:
            j+=1
            c+=str(chaine[j])
            
        encodeur=[(x[:-1],d[x[:-1]]) for x in d.keys() if len(x)>1]

#     if c not in d.keys():
#         d[c]=i
#         encodeur=[(x[:-1],d[x[:-1]]) for x in d.keys() if len(x)>1]
#     else:
#         encodeur=[(x[:-1],d[x[:-1]]) for x in d.keys() if len(x)>1]
#         encodeur.append((c,d[c]))
    return [x[1] for x in encodeur]

def lzw(chaine):
    dic={}
    com=[]
    i=0
    j=1
    cmp=256
    while j<len(chaine):
        if chaine[i:j+1] not in dic:
            dic[chaine[i:j+1]]=cmp
            cmp+=1
            if len(chaine[i:j])==1:
                com.append((ord(chaine[i:j])))
            else:
                com.append((dic[chaine[i:j]]))
            j+=1
            i=j-1
        else:
            j+=1
    if len(chaine[i:j])==1:
        com.append((ord(chaine[i:j])))
    else:
        com.append((dic[chaine[i:j]]))
    return com

def LZWdecoding(code):
    dic=[]
    dic.append([code[0],chr(code[0])])
    decodeur={}
    i=1
    j=256
    while i!=len(code):
        if code[i] <=255:
            dic.append([code[i],chr(code[i])])

        else:
            if code[i] not in decodeur.keys():
                dic.append([code[i],2*chr(code[i-1])])
            else:
                dic.append([code[i],decodeur[code[i]]])
        decodeur[j]=dic[i-1][1]+dic[i][1][0]

        j+=1
        i+=1
#     print(dic)
#     print(decodeur)
    chaine=[x[1] for x in dic]
    return chaine

def convert():
    global nchannels,sampwidth,framerate,framewidth,duration,data,len_data
    src = file_path.get()
    dst="converted/output.irm"
    
    file_path.set(dst)
    (nchannels,sampwidth,framerate,framewidth,duration,data,len_data)=lire_audio(src)
    chaine_compresse,zero_255=preparer(data)
    comprese=lzw(chaine_compresse)
    string=[]
    irm=open(dst,"wb")
    string=[]

    for i in range(len(comprese)):
        irm.write(comprese[i].to_bytes(2,"big"))
        string.append(comprese[i].to_bytes(2,"big"))
    irm.close()
    label2.config(text="Taille Compresse : "+str(os.path.getsize(file_path.get()))+"ko")
    
    
    
    
pygame.mixer.init()
def play_music():
    src = file_path.get()
    nom,extension=os.path.splitext(src)

    if extension=='.irm':
        
        irm1=open(src,"rb")
        lignes=irm1.read(2)
        ho=[]
        while(lignes):
            ho.append(int.from_bytes(lignes, "big"))
            lignes=irm1.read(2)
        irm1.close()
        
        decompres=LZWdecoding(ho)

        v_data=np.frombuffer(data,np.int16)
        sd.play(v_data,framerate)
    else:
        audio=file_path.get()
        pygame.mixer.music.load(audio)
        pygame.mixer.music.play()
    

    
def pause():
    try:
        pygame.mixer.music.pause()
    except Exception as e:
        print(e)

def resume():
    try:
        pygame.mixer.music.unpause()
    except Exception as e:
        print(e)
        

# def convert():

#     src = file_path.get()
#     dst="converted/output.mp3"
    
#     file_path.set(dst)
# #     AudioSegment.from_mp3("mymp3.mp3").export('myogg.ogg', format='ogg')
#     sound = pydub.AudioSegment.from_mp3(src)
#     sound.export(dst,format='wav')
    
#     label2.config(text="Taille Compresse : "+str(os.path.getsize(file_path.get()))+"ko")

    
#Buttons Section
frame=tk.Frame(root)
ouvrir=tk.Button(frame,text="Ouvrir",command=browse_button)
ouvrir.grid(row=0,column=1,ipadx=4)
play=tk.Button(frame,text="Play",command=play_music)
play.grid(row=0,column=2,ipadx=4)
stop=tk.Button(frame,text="Stop",command=pause)
stop.grid(row=0,column=3,ipadx=4)
pause=tk.Button(frame,text="Resume",command=resume)
pause.grid(row=0,column=4,ipadx=4)
# prec=tk.Button(frame,text="Prec")
# prec.grid(row=0,column=5,ipadx=4)
# suiv=tk.Button(frame,text="Suivant")
# suiv.grid(row=0,column=6,ipadx=4)

frame.grid(row=0,column=0,sticky='W',pady=10)

#Chemin Input
file_path=tk.StringVar();
chemin=tk.Entry(root,textvariable=file_path,width=50)
chemin.grid(row=0,column=1,padx=50)

#chemin input disable
chemindisable1=tk.Entry(root,textvariable=file_path,state='disabled')
chemindisable1.grid(row=1,column=0,sticky=tk.W+tk.E,pady=10)

#scrollbar
# scroll=tk.Scrollbar(root,orient='horizontal')
# scroll.grid(row=2,column=0,sticky=tk.W+tk.E)

#chemin button
# cheminbutton=tk.Button(text='chemin de fichier')
# cheminbutton.grid(row=2,column=1,pady=15)


img = ImageTk.PhotoImage(Image.open(r"cn.jpg"))
imgcontainer = tk.Label(root, image = img)
imgcontainer.grid(row=3,column=0)

#compression button section
frame2=tk.Frame(root)
separator=ttk.Separator(frame2,orient='horizontal')
separator.grid(row=0,column=0,pady=5,sticky='WE')
compressbutton=tk.Button(frame2,text="CONVERT TO IRM",command=convert,bg='#997fad', fg='#ffffff',font=('bold'))
compressbutton.grid(row=1,column=0,pady=5,sticky='WE')
separator=ttk.Separator(frame2,orient='horizontal')
separator.grid(row=2,column=0,pady=5,sticky='WE')
# ouvrircompressbutton=tk.Button(frame2,text="Ouvrir fichier compresse")
# ouvrircompressbutton.grid(row=2,column=0,ipadx=15)

frame2.grid(row=3,column=1)

#Info Section
frame3=tk.Frame(root)
label1=tk.Label(frame3,text="Taille D'origine : " )
label1.grid(row=0,column=0,sticky='W')
label2=tk.Label(frame3,text="Taille Compresse : " )
label2.grid(row=1,column=0,sticky='W')
label3=tk.Label(frame3,text="Taux de Compression : " )
label3.grid(row=2,column=0,sticky='W')
label4=tk.Label(frame3,text="Temp de Compression : " )
label4.grid(row=3,column=0,sticky='W')

frame3.grid(row=4,column=0,sticky='W')

root.mainloop()

pygame 2.0.2 (SDL 2.0.16, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\users\lenovo\appdata\local\programs\python\python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\lenovo\appdata\local\programs\python\python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Lenovo\AppData\Local\Temp/ipykernel_5664/708644056.py", line 146, in convert
    (nchannels,sampwidth,framerate,framewidth,duration,data,len_data)=lire_audio(src)
  File "C:\Users\Lenovo\AppData\Local\Temp/ipykernel_5664/708644056.py", line 29, in lire_audio
    f=AudioSegment.from_file(file=audio,format=extension[1:])
  File "c:\users\lenovo\appdata\local\programs\python\python39\lib\site-packages\pydub\audio_segment.py", line 651, in from_file
    file, close_file = _fd_or_path_or_tempfile(fi

In [ ]:
import pyglet
song = pyglet.media.load('thesong.ogg')
song.play()
pyglet.app.run()

In [ ]:
import numpy as np
import simpleaudio as sa

frequency = 440  # Our played note will be 440 Hz
fs = 44100  # 44100 samples per second
seconds = 3  # Note duration of 3 seconds

# Generate array with seconds*sample_rate steps, ranging between 0 and seconds
t = np.linspace(0, seconds, seconds * fs, False)

# Generate a 440 Hz sine wave
note = np.sin(frequency * t * 2 * np.pi)

# Ensure that highest value is in 16-bit range
audio = note * (2**15 - 1) / np.max(np.abs(note))
# Convert to 16-bit data
audio = audio.astype(np.int16)

# Start playback
play_obj = sa.play_buffer(audio, 1, 2, fs)

# Wait for playback to finish before exiting
play_obj.wait_done()
